In [841]:
#  Importing Necessary Library
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from random import random,seed
from random import randrange
from sklearn.metrics import accuracy_score,confusion_matrix,cohen_kappa_score
from math import exp
from scipy.special import softmax
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
import os
import matplotlib.pyplot as plt

In [285]:
df=pd.read_csv("dataset/iris.csv")

In [286]:
seed(1)

In [855]:
# Load a CSV file
def loadCsv(filename):
        trainset=pd.read_csv(filename)
        return trainset

# Normalization

def normalize(dataset):
    min_max_obj=MinMaxScaler()
    transformed_data=min_max_obj.fit_transform(dataset)
    return transformed_data

# Convert column values to float
def column_to_float(dataset, column):
        return dataset[column].astype(float)

#  Label encoding target variable

def encode_target(dataset,target_column):
        label_encoder=LabelEncoder()
        label_encoded=label_encoder.fit_transform(dataset.iloc[:,target_column])
        with open("artifacts/label_encoder.pickle","wb") as s:
                pickle.dump(label_encoder,s)
        print("Variable encoded")
        return label_encoded

# Method to create cross validation splits

def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = dataset.to_numpy().tolist()
        fold_size = int(len(dataset_copy) / n_folds)
        for i in range(n_folds):
                fold = list()
                while len(fold) < fold_size:
                        index = randrange(len(dataset_copy))
                        fold.append(dataset_copy.pop(index))
                dataset_split.append(fold)
        print("Cross validation split done")
        return dataset_split

#  Method to Create a neural network

# Creating Neural network with 1 hidden layes
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)],'prev':[0 for i in range(n_hidden+1)]} for i in range(n_outputs)]
        network.append(output_layer)
        # initialize parameters of activation function g(x) = k0 + k1x
        k=[random(),random()]
        return network,k


# Calculate neuron activation for an input
def activate(weights, inputs):
        # print("Activate func")
        # print(weights)
        # print(inputs)
        activation = weights[-1]   # Weights[-1]  is Bias term
        for i in range(len(weights)-1):
                activation += weights[i] * inputs[i]
        return activation


# Transfer neuron activation     g(x) = k0 + k1x
def transfer(activation,K):
        return K[0]+K[1]*(activation)


# Forward propagate input to a network output  -  Hidden Layers use Ada Acti Activation (g(x) = k0 + k1x) and output layer use Softmax
def forward_propagate(network, row,K):
        inputs = row
        val={}
        for index,layer in enumerate(network):
                if index!=len(network)-1:
                        new_inputs = []
                        z=[]
                        for neuron in layer:
                                activation = activate(neuron['weights'], inputs)
                                z.append(activation)
                                # print(f"Hidden Layer {index} ,Applying Ada Act Activation")
                                neuron['output'] = transfer(activation,K)
                                new_inputs.append(neuron['output'])
                        val[f"Z{index+1}"]=np.array(z)
                        val[f"A{index+1}"]=np.array(new_inputs)
                        inputs = new_inputs
                else:
                        # print("Final output layer : Applying Softmax")
                        z=[]
                        new_inputs = []
                        for neuron in layer:
                                activation = activate(neuron['weights'], inputs)
                                z.append(activation)
                                neuron['output'] = activation
                                new_inputs.append(neuron['output'])
                        val[f"Z{index+1}"]=np.array(z)
                        inputs = softmax(new_inputs,axis=0)
                        val[f"A{index+1}"]=np.array(inputs)   
                        
        return inputs,val





# Backpropagate error and store in neurons
def backward_propagate_error(network,X_row,expected,val,yhat,K):
        # print("Back propagate Error started")
        derivatives={}
        params={}
        W2=[]   #  weights in final layer                                                        # 3x3
        for i in network[1]:
                W2.append(i["weights"][:-1])

        W1=[]   #  weights in 1st hidden layer  layer                                            # 3x3
        for i in network[0] : # weights in first hiddent layer
                W1.append(i["weights"][:-1])

        B2=[]   #  Bias in final layer                                                           #1x3
        for i in network[1]:
                B2.append(i["weights"][-1])

        B1=[]    #  Bias in hidden  layer                                                         #1x3
        for i in network[0] : # weights in first hiddent layer
                B1.append(i["weights"][-1])

        W2=np.array(W2)
        W1=np.array(W1)
        B2=np.array(B2)
        B1=np.array(B1)
        K=np.array(K)


        params["W2"]=W2
        params["W1"]=W1
        params["B2"]=B2
        params["B1"]=B1
        params["K"]=K

        
        # dz3 = a3 − y  where a3 is output from final layer after softmax activation
        dZ2=  np.expand_dims(np.array(expected)-np.array(yhat),axis=0)
                                                                      #1x3
        #  dw3 = (1/t) aT2 × dz3
        dW2=np.dot(np.expand_dims(val["A1"],axis=0).T,dZ2)                                                       #3x3


        # db3 = avgcol(dz3)

        dB2=np.mean(dZ2,axis=0)

        # da1 = dz2 × w2T      w2 is the weights in final layer
        dA1=np.dot(dZ2,np.array(W2))                                                                

        # dz1 = g' (z1) ∗ da1


        dZ1= np.array(K[1]*dA1)

        # dw1 = 1/t a0.T × dz1  a0 is the input given to neural  network
        dW1=np.dot(dZ1.T,np.expand_dims(np.array(X_row),axis=1))

        #  db1 = avgcol(dz1)
        dB1=np.mean(dZ1,axis=1)
        # print("dB1 shape",dB1.shape)


        #  Calculating K derivatives

        dK=[np.mean(dA1)]
        dK.append(np.mean(np.multiply(dA1,val["Z1"])))
        dk=np.array(dK)
        # print("dK",dK)
        derivatives["dW1"]=dW1
        derivatives["dW2"]=dW2
        derivatives["dB1"]=dB1
        derivatives["dB2"]=dB2
        derivatives["dK"]=dK

        return derivatives,params


def update_params(network,alpha, derivatives, params):
    W1 = params["W1"]
    W2 = params["W2"]
    B1 = params["B1"]
    B2 = params["B2"]
    K = params["K"]
    
    dW1 = derivatives["dW1"]
    dW2 = derivatives["dW2"]
    dB1 = derivatives["dB1"]
    dB2 = derivatives["dB2"]
    dK = derivatives["dK"]
    



    params["W1"] = W1-(alpha*(dW1))
    params["W2"] = W2-(alpha*(dW2))
    
    params["B1"] = B1-(alpha*(dB1))
    params["B2"] = B2-(alpha*(dB2))
    params["K"] = K-(alpha*np.array(dK))

    new_hl_weights=np.concatenate((params["W1"],params["B1"].reshape(3,-1)),axis=1).tolist()
    new_h2_weights=np.concatenate((params["W2"],params["B2"].reshape(3,-1)),axis=1).tolist()

    for ac_layer,up_layer in zip(network,(new_hl_weights,new_h2_weights)):
        for neuron,new_weights in zip(ac_layer,up_layer):
                neuron["weights"]=new_weights

    return params
                
    



# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs,K,fold):
        target_col=train.columns[-1]
        acc_epoch={}
        for epoch in range(n_epoch):
                act=[]
                out=[]
                for index in train.index:
                        row=train.loc[index,:]
                        outputs,val = forward_propagate(network, row,K)
                        #print(network)
                        expected = [0 for i in range(n_outputs)]
                        # print("Row",row[-1])
                        # print("Expected",expected)
                        expected[int(row[-1])] = 1
                        #print("expected row{}\n".format(expected))
                        derivatives,params= backward_propagate_error(network,train.loc[[index],target_col],expected,val,outputs,K)
                        # print(f"Epoch {epoch}")
                        params=update_params(network,l_rate,derivatives,params)
                        K=params["K"]
                        act.append(int(train.loc[index,train.columns[-1]]))
                        out.append(predict(network,row,K))
                acc_epoch[f"epoch{epoch}"]=accuracy_score(act,out)

        acc_epoch=pd.DataFrame(acc_epoch,index=[0])
        acc_epoch.to_csv(f"artifacts/acc_fold_{fold}.csv",index=False)

                


        return K

# Make a prediction with a network
def predict(network, row,k):
        outputs,_ = forward_propagate(network, row,k)
        # print(outputs)
        return np.argmax(outputs)

def loss(Y_hat, Y):
    N = len(Y)
    ce = -np.sum(np.multiply(Y, np.log(Y_hat))) / N
    return ce


def back_propagation(train, test, l_rate, n_epoch, n_hidden,fold):
        n_inputs = len(train.columns) - 1
        n_outputs = len(train[train.columns[-1]].unique())
        network,K = initialize_network(n_inputs, n_hidden, n_outputs)

        print("initial K",K)
        K=train_network(network, train, l_rate, n_epoch, n_outputs,K,fold)
        #print("network {}\n".format(network))
        predictions = list()
        for row in test.index:
                prediction = predict(network, test.loc[row,:],K)
                predictions.append(prediction)
        print("Final K",K)
        # vars()[f"Final_k_{fold}"]=K
        return(predictions,K,network)


# Evaluate an algorithm using a cross validation split
def run_algorithm(dataset, algorithm, n_folds, l_rate, n_epoch, n_hidden):
        
        folds = cross_validation_split(dataset, n_folds)
        #for fold in folds:
                #print("Fold {} \n \n".format(fold))
        acc_scores = {}
        prec_score={}
        rec_score={}
        specificity={}
        sensitivity={}
        cohen_kappa={}
        # loss_val={}
        for index,fold in enumerate(folds):
                #print("Test Fold {} \n \n".format(fold))
                train_set = list(folds)
                train_set.remove(fold)
                train_set = sum(train_set, [])
                test_set = list()
                for row in fold:
                        row_copy = list(row)
                        test_set.append(row_copy)
                        # row_copy[-1] = None
                train_set=pd.DataFrame(train_set,columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'])
                test_set=pd.DataFrame(test_set,columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'])
                # print("Gonna start back prop algo")
                predicted,K,network = algorithm(train_set, test_set,l_rate, n_epoch, n_hidden,index)
                # print("back propagation algo done")
                actual = np.array([row[-1] for row in fold])
                
                accuracy = accuracy_score(actual, predicted)

                cm = confusion_matrix(actual, predicted)
                print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in cm]))
                #confusionmatrix = np.matrix(cm)
                FP = cm.sum(axis=0) - np.diag(cm)
                FN = cm.sum(axis=1) - np.diag(cm)
                TP = np.diag(cm)
                TN = cm.sum() - (FP + FN + TP)
                print('False Positives\n {}'.format(FP))
                print('False Negetives\n {}'.format(FN))
                print('True Positives\n {}'.format(TP))
                print('True Negetives\n {}'.format(TN))
                TPR = TP/(TP+FN)
                print('Sensitivity \n {}'.format(TPR))
                TNR = TN/(TN+FP)
                print('Specificity \n {}'.format(TNR))
                Precision = TP/(TP+FP)
                print('Precision \n {}'.format(Precision))
                Recall = TP/(TP+FN)
                print('Recall \n {}'.format(Recall))
                Acc = (TP+TN)/(TP+TN+FP+FN)
                print('Áccuracy \n{}'.format(Acc))
                Fscore = 2*(Precision*Recall)/(Precision+Recall)
                print('FScore \n{}'.format(Fscore))
                coh=cohen_kappa_score(actual, predicted)
                print('Çohen Kappa \n{}'.format(coh))

                acc_scores[f"fold{index}"]=accuracy
                prec_score[f"fold{index}"]=Precision
                rec_score[f"fold{index}"]=Recall
                specificity[f"fold{index}"]=TNR
                sensitivity[f"fold{index}"]=TPR
                cohen_kappa[f"fold{index}"]=coh

        return acc_scores,prec_score,rec_score,specificity,sensitivity,cohen_kappa

In [856]:
seed(1)
filename = 'dataset/iris.csv'
df = pd.read_csv(filename)

#  convert independent variables to float
for i in df.columns[:-1]:
        df[i]=column_to_float(df,i)

# Label Encoding target variable
df["species"]=encode_target(df,4)

# Normalize the independent variable

df.iloc[:,:4]=normalize(df.iloc[:,:4])

# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 30
n_hidden = 3
acc_scores,prec_score,rec_score,specificity,sensitivity,cohen_kappa = run_algorithm(df, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

# print(acc_scores)

print('Scores: %s' % acc_scores)
print('Mean Accuracy: %.3f%%' % (sum(acc_scores.values())/float(len(acc_scores))))

Variable encoded
Cross validation split done
initial K [0.18803930475131292, 0.10876169244541334]


C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:63: RuntimeWarning: overflow encountered in double_scalars
  activation += weights[i] * inputs[i]
c:\Users\Vijay\anaconda3\envs\dl\lib\site-packages\scipy\special\_logsumexp.py:214: RuntimeWarning: invalid value encountered in subtract
  return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


Final K [nan nan]
  11   0   0
   8   0   0
  11   0   0
False Positives
 [19  0  0]
False Negetives
 [ 0  8 11]
True Positives
 [11  0  0]
True Negetives
 [ 0 22 19]
Sensitivity 
 [1. 0. 0.]
Specificity 
 [0. 1. 1.]
Precision 
 [0.36666667        nan        nan]
Recall 
 [1. 0. 0.]
Áccuracy 
[0.36666667 0.73333333 0.63333333]
FScore 
[0.53658537        nan        nan]
Çohen Kappa 
0.0
initial K [0.30638662033324593, 0.8585144063565593]


C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:338: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)
C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:69: RuntimeWarning: overflow encountered in double_scalars
  return K[0]+K[1]*(activation)
c:\Users\Vijay\anaconda3\envs\dl\lib\site-packages\scipy\special\_logsumexp.py:214: RuntimeWarning: invalid value encountered in subtract
  return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


Final K [nan nan]
  10   0   0
  11   0   0
   9   0   0
False Positives
 [20  0  0]
False Negetives
 [ 0 11  9]
True Positives
 [10  0  0]
True Negetives
 [ 0 19 21]
Sensitivity 
 [1. 0. 0.]
Specificity 
 [0. 1. 1.]
Precision 
 [0.33333333        nan        nan]
Recall 
 [1. 0. 0.]
Áccuracy 
[0.33333333 0.63333333 0.7       ]
FScore 
[0.5 nan nan]
Çohen Kappa 
0.0
initial K [0.8716215074235552, 0.8996782696347811]


...

C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:338: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)
C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:63: RuntimeWarning: overflow encountered in double_scalars
  activation += weights[i] * inputs[i]
C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:63: RuntimeWarning: invalid value encountered in double_scalars
  activation += weights[i] * inputs[i]


Final K [nan nan]
  10   0   0
  10   0   0
  10   0   0
False Positives
 [20  0  0]
False Negetives
 [ 0 10 10]
True Positives
 [10  0  0]
True Negetives
 [ 0 20 20]
Sensitivity 
 [1. 0. 0.]
Specificity 
 [0. 1. 1.]
Precision 
 [0.33333333        nan        nan]
Recall 
 [1. 0. 0.]
Áccuracy 
[0.33333333 0.66666667 0.66666667]
FScore 
[0.5 nan nan]
Çohen Kappa 
0.0
initial K [0.09168312261651779, 0.1151024984279273]


C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:338: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)
C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:69: RuntimeWarning: overflow encountered in double_scalars
  return K[0]+K[1]*(activation)
C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:63: RuntimeWarning: overflow encountered in double_scalars
  activation += weights[i] * inputs[i]
C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:63: RuntimeWarning: invalid value encountered in double_scalars
  activation += weights[i] * inputs[i]


Final K [nan nan]
  13   0   0
  11   0   0
   6   0   0
False Positives
 [17  0  0]
False Negetives
 [ 0 11  6]
True Positives
 [13  0  0]
True Negetives
 [ 0 19 24]
Sensitivity 
 [1. 0. 0.]
Specificity 
 [0. 1. 1.]
Precision 
 [0.43333333        nan        nan]
Recall 
 [1. 0. 0.]
Áccuracy 
[0.43333333 0.63333333 0.8       ]
FScore 
[0.60465116        nan        nan]
Çohen Kappa 
0.0
Scores: {'fold0': 0.36666666666666664, 'fold1': 0.3333333333333333, 'fold2': 0.2, 'fold3': 0.3333333333333333, 'fold4': 0.43333333333333335}
Mean Accuracy: 0.333%


C:\Users\Vijay\AppData\Local\Temp\ipykernel_14828\3506005137.py:338: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)


In [851]:
print("Mean accuracy scores for all 5 folds for 30 epochs")
print(acc_scores)

{'fold0': 0.36666666666666664,
 'fold1': 0.3333333333333333,
 'fold2': 0.2,
 'fold3': 0.3333333333333333,
 'fold4': 0.43333333333333335}